In [ ]:
data = cPickle.load(bz2.BZ2File('./data/homophones.pbz2', 'rb'))

In [3]:
data = [item for value in tqdm(data) for item in list(product(value, repeat=2))]
data = pd.DataFrame(data, columns=['in', 'out'])
data.drop_duplicates(inplace=True)
data.dropna(inplace=True)
data.head()

100%|██████████| 123000/123000 [00:06<00:00, 18549.82it/s]


,in,out
0,AEA,AEA
1,AEA,AAA
2,AEA,A
3,AEA,AU
4,AEA,AWA


In [3]:
# load config, if exists
config = None
folder_path = './checkpoint/'

if os.path.isfile(folder_path + 'config.pkl'):
    print('loading the config file..')
    with open(folder_path + 'config.pkl', 'rb') as fp:
        config = pickle.load(fp)

loading the config file..


In [6]:
if not config:
    max_sent_len = 0
    max_output_len = 0

    for inp, out in data.values:
        max_sent_len = max(max_sent_len, len(inp))
        max_output_len = max(max_output_len, len(out))

    print(
        "original parameters - max_sent_len = {}, max_output_len = {}".format(
            max_sent_len, max_output_len
        )
    )

    max_sent_len = min(50, max_sent_len)
    max_output_len = min(50, max_output_len)

    print(
        "updated parameters - max_sent_len = {}, max_output_len = {}".format(
            max_sent_len, max_output_len
        )
    )

else:
    print("loading params from config file..")

    max_sent_len = config["max_sent_len"]
    max_output_len = config["max_output_len"]

    print(
        "parameters: max_sent_len = {}, max_output_len = {}".format(
            max_sent_len, max_output_len
        )
    )

loading params from config file..
parameters: max_sent_len = 37, max_output_len = 37


In [ ]:
data.drop_duplicates(inplace=True)

data = data.sample(frac=1)
data.reset_index(inplace=True, drop=True)

data = data.apply(lambda x: x.astype(str) \
                            .str.upper() \
                            .str.replace(r'[^A-Z0-9\.%/\s]', '') \
                            .str.replace(r"\xa0", " ") \
                            .str.strip())


data['in'] = [v[:max_sent_len] for v in data['in']]
data['out'] = [v[:max_output_len-2] for v in data['out']]

data = data[data['in'] != '']
data.head()

/tmp/ipykernel_2324276/2367742055.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  data = data.apply(lambda x: x.astype(str) \


In [3]:
class CDataset(Dataset):
    def __init__(self, data, config=None):
        self.data = data

        if config is None:
            self.max_output_len = (
                max([len(sent) for sent in self.data["out"].values]) + 2
            )
            
            self.max_sent_len = max([len(sent) for sent in self.data["in"].values])

            self.char2idx, self.idx2char = self.char2index(
                set(pd.concat([data["in"], data["out"]]))
            )

        else:
            self.max_sent_len = config["max_sent_len"]
            self.max_output_len = config["max_output_len"]

            self.char2idx = config["char2idx"]
            self.idx2char = {v: k for k, v in self.char2idx.items()}

    def __len__(self):
        return len(self.data)

    def char2index(self, data=[]):
        data = set([char for sent in data for char in sent])

        char2idx = {c: i + 3 for i, c in enumerate(data)}
        char2idx.update({"<PAD>": 0, "<SOS>": 1, "<EOS>": 2})

        idx2char = {v: k for k, v in char2idx.items()}
        return char2idx, idx2char

    def __getitem__(self, idx):
        # output
        label = np.asarray(
            [self.char2idx["<SOS>"]]
            + [self.char2idx[x] for x in self.data["out"].values[idx]]
            + [self.char2idx["<EOS>"]]
        )
        label = torch.LongTensor(
            np.pad(
                label,
                (self.char2idx["<PAD>"], self.max_output_len - len(label)),
                "constant",
            )
        )

        # word input and padding
        word = np.asarray([self.char2idx[char] for char in self.data["in"].values[idx]])
        word = torch.LongTensor(
            np.pad(
                word,
                (self.char2idx["<PAD>"], self.max_sent_len - len(word)),
                "constant",
            )
        )

        return word, label

In [ ]:
from torch.utils.data import DataLoader, random_split

# 700
data.reset_index(drop=True, inplace=True)
BATCH_SIZE = 50000
cdata = CDataset(data, config)

# train test split
train_split_ratio = 0.8

train_split = int(train_split_ratio * len(cdata))
# test_split = max(len(cdata) - train_split, 1000)
test_split = len(cdata) - train_split
# train_split = int(train_split_ratio * len(cdata)) - test_split

cdata_train, cdata_test = random_split(cdata, [train_split, test_split])

# train dataloader
train_dataloader_args =  dict(shuffle=True, batch_size=BATCH_SIZE, num_workers=0)
trainloader = DataLoader(cdata_train, pin_memory=True, **train_dataloader_args)

# test dataloader
test_dataloader_args =  dict(shuffle=True, batch_size=BATCH_SIZE, num_workers=0)
testloader = DataLoader(cdata_test, pin_memory=True, **test_dataloader_args)

In [ ]:
if not os.path.isfile(folder_path + 'config.pkl') :
    config = dict()
  
    config.update({'max_sent_len':cdata.max_sent_len, 
                   'max_output_len':cdata.max_output_len, 
                   'char2idx':cdata.char2idx})
    
    with open(folder_path + 'config.pkl', 'wb') as fp:
        pickle.dump(config, fp)

In [4]:
class Embedding(nn.Module):
    def __init__(self, vocab_size, dimension):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, dimension, padding_idx=0)
        self.dimension = dimension

    def forward(self, input_vec):
        return self.embedding(input_vec) * math.sqrt(self.dimension)


class PositionalEncoding(nn.Module):
    def __init__(self, max_seq_len, dimension, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        positional_enc = torch.zeros(max_seq_len, dimension)

        den = torch.pow(
            10000, torch.div(torch.arange(0, dimension / 2) * 2, float(dimension))
        )
        num = torch.arange(0, max_seq_len).unsqueeze(1)

        positional_enc[:, 0::2], positional_enc[:, 1::2] = (
            torch.sin(num / den),
            torch.cos(num / den),
        )
        positional_enc = positional_enc.unsqueeze(0)
        self.register_buffer("positional_enc", positional_enc)

    def forward(self, input_vec):
        seq_len = input_vec.size(1)
        return self.dropout(input_vec + Variable(self.positional_enc[:, :seq_len]))


class MultiHeadedAttention(nn.Module):
    def __init__(self, dimension, heads, dropout=0.0):
        super().__init__()
        self.heads = heads
        self.dimension = dimension
        self.queryl = nn.Linear(dimension, dimension)
        self.keyl = nn.Linear(dimension, dimension)
        self.valuel = nn.Linear(dimension, dimension)
        self.outl = nn.Linear(dimension, dimension)
        self.dropout = nn.Dropout(dropout)

    def forward(self, query, key, value, mask=None):

        assert self.dimension == query.size(-1)
        batch_size = query.size(0)

        query = self.queryl(query)
        key = self.keyl(key)
        value = self.valuel(value)

        query = query.view(
            batch_size, -1, self.heads, query.size(-1) // self.heads
        ).transpose(1, 2)
        key = key.view(
            batch_size, -1, self.heads, key.size(-1) // self.heads
        ).transpose(1, 2)

        value = value.view(
            batch_size, -1, self.heads, value.size(-1) // self.heads
        ).transpose(1, 2)


        attn = self.attention(query, key, value, mask, self.dropout)


        concat = attn.transpose(1, 2).reshape(
            batch_size, -1, query.size(-1) * self.heads
        )

        return self.outl(concat)


    def attention(self, query, key, value, mask=None, dropout=None):
        qk = torch.div(
            torch.matmul(query, key.transpose(-2, -1)), math.sqrt(query.size(-1))
        )

        if mask is not None:
            mask = mask.unsqueeze(1)
            qk = qk.masked_fill(mask == 0, -1e9)

        qk = nn.Softmax(dim=-1)(qk)
        qk = self.dropout(qk) if dropout is not None else qk
        return torch.matmul(qk, value)


class FeedForwardNet(nn.Module):
    def __init__(self, dimension, dff=2048, dropout=0.1):
        super().__init__()
        self.l = nn.Linear(dimension, dff)
        self.out = nn.Linear(dff, dimension)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_vec):
        return self.out(self.dropout(F.relu(self.l(input_vec))))


class LayerNorm(nn.Module):
    def __init__(self, dimension, delta=1e-6):
        super().__init__()
        self.gain = nn.Parameter(torch.ones(dimension))
        self.bias = nn.Parameter(torch.zeros(dimension))
        self.delta = delta

    def forward(self, input_vec):
        mean = torch.mean(input_vec, dim=-1, keepdim=True)
        std = torch.std(input_vec, dim=-1, keepdim=True) + self.delta
        return (self.gain / std) * (input_vec - mean) + self.bias


class ResidualConnection(nn.Module):
    def __init__(self, dimension, dropout=0.1):
        super().__init__()
        self.norm = LayerNorm(dimension)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_vec, sublayer):
        return input_vec + self.dropout(sublayer(self.norm(input_vec)))


class EncoderLayer(nn.Module):
    def __init__(self, dimension, head=8, dropout=0.1):
        super().__init__()
        self.attn = MultiHeadedAttention(dimension, head, dropout)
        self.ffnn = FeedForwardNet(dimension, dropout=dropout)
        self.resconn1 = ResidualConnection(dimension, dropout)
        self.resconn2 = ResidualConnection(dimension, dropout)

        self.norm = LayerNorm(dimension)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_vec, mask=None):
        attn = self.resconn1(input_vec, lambda x: self.attn(x, x, x, mask))
        return self.resconn2(attn, self.ffnn), attn


class Encoder(nn.Module):
    def __init__(
        self, vocab_size, number_of_layers, head, max_seq_len, dimension, dropout
    ):
        super().__init__()
        self.emb = Embedding(vocab_size, dimension)
        self.penc = PositionalEncoding(max_seq_len, dimension, dropout)
        self.enclays = nn.ModuleList(
            [
                copy.deepcopy(EncoderLayer(dimension, head, dropout))
                for _ in range(number_of_layers)
            ]
        )
        self.norm = LayerNorm(dimension)

    def forward(self, input_vec, mask=None):
        # input_vec - batch_size, max_sent_len
        emb = self.emb(input_vec) # emb size = batch, max_sent_len, dimension
        emb = self.penc(emb)

        for layer in self.enclays:
            emb, _ = layer(emb, mask)
        emb = self.norm(emb)
        return emb, _


class DecoderLayer(nn.Module):
    def __init__(self, dimension, heads=8, dropout=0.1):
        super().__init__()
        self.ffnn = FeedForwardNet(dimension, dropout=dropout)
        self.resconn = nn.ModuleList(
            [copy.deepcopy(ResidualConnection(dimension, dropout)) for _ in range(3)]
        )
        self.attn = nn.ModuleList(
            [
                copy.deepcopy(MultiHeadedAttention(dimension, heads, dropout))
                for _ in range(2)
            ]
        )

    def forward(self, input_vec, encoder_output, encmask, decmask):
        selfattn = self.resconn[0](input_vec, lambda x: self.attn[0](x, x, x, decmask))

        encdecattn = self.resconn[1](
            selfattn, lambda x: self.attn[1](x, encoder_output, encoder_output, encmask)
        )

        return self.resconn[2](encdecattn, self.ffnn)


class Decoder(nn.Module):
    def __init__(
        self, vocab_size, number_of_layers, head, max_seq_len, dimension, dropout
    ):
        super().__init__()
        self.emb = Embedding(vocab_size, dimension)
        self.penc = PositionalEncoding(max_seq_len, dimension, dropout)
        self.declays = nn.ModuleList(
            [
                copy.deepcopy(DecoderLayer(dimension, head, dropout))
                for i in range(number_of_layers)
            ]
        )
        self.norm = LayerNorm(dimension)

    def forward(self, input_vec, encoder_output, encmask, decmask):

        emb = self.emb(input_vec)
        emb = self.penc(emb)

        for layer in self.declays:
            emb = layer(emb, encoder_output, encmask, decmask)
        return self.norm(emb)


class Transformer(nn.Module):
    def __init__(
        self,
        envocab_size,
        devocab_size,
        enc_max_seq_len,
        dec_max_seq_len,
        head,
        number_of_enc_layers,
        number_of_dec_layers,
        dimension,
        dropout,
    ):
        super().__init__()
        self.encoder = Encoder(
            envocab_size, number_of_enc_layers, head, enc_max_seq_len, dimension, dropout
        )
        self.decoder = Decoder(
            devocab_size, number_of_dec_layers, head, dec_max_seq_len, dimension, dropout
        )
        self.ffnn = nn.Linear(dimension, devocab_size)

    def forward(self, enc_input_vec, dec_input_vec, encmask, decmask):

        encout, _ = self.encoder(enc_input_vec, encmask)
        decout = self.decoder(dec_input_vec, encout, encmask, decmask)

        return F.log_softmax(self.ffnn(decout), dim=-1)


class Batch:
    def __init__(self, src, trg=None, device='cpu', pad=0):
        self.src = src
        self.src_mask = (src!=pad).unsqueeze(1)


        if trg is not None:
            self.trg = trg[:, :-1]
            self.trg_y = trg[:, 1:]
            self.ntokens = (self.trg_y != pad).data.sum()

            trg_mask = (self.trg != pad).unsqueeze(-2)
            dimension = self.trg.size(-1)

            mask = torch.tril(torch.ones(1, dimension, dimension, device=device))
            mask[mask != 0] = 1
            mask = Variable(mask > 0)

            if self.trg.is_cuda:
                mask.cuda()
            self.trg_mask = trg_mask & mask
 

class LabelSmoothing(nn.Module):
    def __init__(self, vocab_size, pad_index, alpha):
        super().__init__()
        self.alpha = alpha
        self.vocab_size = vocab_size
        self.pad_index = pad_index

    def forward(self, prediction, target):
        prediction = prediction.contiguous().view(-1, prediction.size(-1))
        target = target.contiguous().view(-1)

        one_hot_target = torch.nn.functional.one_hot(target, num_classes=prediction.size(-1))
        one_hot_target[:, self.pad_index] = 0
        one_hot_target = (one_hot_target * (1 - self.alpha)) + (
            self.alpha / (self.vocab_size - 2)
        )
        one_hot_target.masked_fill_((target == self.pad_index).unsqueeze(1), 0)

        return F.kl_div(prediction, one_hot_target, reduction="sum")

    
class CustomAdam:
    def __init__(self, dimension, optimizer, warmup_steps=4000, step_num=0):
        self.optimizer = optimizer
        self.step_num = step_num
        self.dimension = dimension
        self.warmup_steps = warmup_steps

    def step(self):
        self.step_num += 1
        lr = self.rate()

        for pg in self.optimizer.param_groups:
            pg["lr"] = lr

        self.optimizer.step()

    def rate(self):
        return self.dimension ** (-0.5) * min(
            self.step_num ** (-0.5), self.step_num * self.warmup_steps ** (-1.5)
        )

        
def init_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

In [5]:
# new network definition
INPUT_DIM = len(config["char2idx"])
OUTPUT_DIM = len(config["char2idx"])
HID_DIM = 64
LAYERS = 2
HEADS = 8
DROPOUT = 0.1

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

model = Transformer(INPUT_DIM, OUTPUT_DIM, config["max_sent_len"], config["max_output_len"], HEADS, LAYERS, LAYERS, HID_DIM, DROPOUT).to(device)
model = torch.nn.DataParallel(model)

# load or initialise weights of the model
if os.path.isfile(folder_path + 'model.pt'):
    checkpoint = torch.load(folder_path + 'model.pt', map_location='cpu')
    model.load_state_dict(checkpoint['model_state_dict'] if 'model_state_dict' in checkpoint else checkpoint, strict=False)
    print('loading the new model to train further..')
    
else:
    checkpoint = {"epoch": 0}
    model.apply(init_weights)
    print('new model weights initialised..')

loading the new model to train further..


In [6]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,164,253 trainable parameters


In [7]:
optim = torch.optim.Adam(
            model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9
        )

if checkpoint and 'optimizer_state_dict' in checkpoint:
    print('loading optimizer state from previous run..')
    
    optim.load_state_dict(checkpoint['optimizer_state_dict'])

optimizer = CustomAdam(
        dimension=64,
        warmup_steps=400,
        optimizer=optim,
        step_num=checkpoint['step_num'] if 'step_num' in checkpoint else 0,
    )

criterion = LabelSmoothing(vocab_size=INPUT_DIM, pad_index=0, alpha=0.1)

loading optimizer state from previous run..


In [8]:
from tqdm import tqdm


def train(epoch, model, device, trainloader, optimizer):
    model.train()

    running_loss = 0
    running_accuracy = 0

    correct = 0
    processed = 0
    pbar = tqdm(trainloader)

    for i, (src, trg) in enumerate(pbar, 0):

        src, trg = src.to(device), trg.to(device)
        batcher = Batch(src, trg, device, 0)

        output = model(batcher.src, batcher.trg, batcher.src_mask, batcher.trg_mask)

        loss = criterion(output, batcher.trg_y) / batcher.ntokens
        loss.backward()

        optimizer.step()
        optimizer.optimizer.zero_grad()

        pred = output.argmax(-1)
        labels = batcher.trg_y

        correct += pred.eq(labels.view_as(pred)).sum().item()
        processed += torch.count_nonzero(labels).item()

        # tensorboard writing every 200 mini-batches
        running_loss += loss.detach().item()
        running_accuracy += round((correct / processed) * 100, 14)

        # tqdm writing
        pbar.set_description(
            desc="Train Epoch - {epoch}, Mini Batch - {batch}, Train Accuracy - {accuracy}, Train Loss - {loss}".format(
                epoch=epoch + 1,
                batch=i + 1,
                accuracy=round((correct / processed) * 100, 14),
                loss=round(loss.detach().item(), 4),
            )
        )
        
    running_loss /= len(trainloader.dataset)  
    
    return round((correct / processed) * 100, 14), running_loss

In [ ]:
def test(model, device, testloader):
    model.eval()

    test_loss = 0
    correct = 0
    processed = 0
    pbar = tqdm(testloader)

    with torch.no_grad():
        for i, (src, trg) in enumerate(pbar, 0):
            src, trg = src.to(device), trg.to(device)
            batcher = Batch(src, trg, device, 0)

            output = model(batcher.src, batcher.trg, batcher.src_mask, batcher.trg_mask)

            loss = criterion(output, batcher.trg_y) / batcher.ntokens
            test_loss += loss.detach().item()

            pred = output.argmax(-1)
            labels = batcher.trg_y

            correct += pred.eq(labels.view_as(pred)).sum().item()
            processed += torch.count_nonzero(labels).item()

            pbar.set_description(
                desc="Test Accuracy - {accuracy}, Test Loss - {loss}".format(
                    accuracy=round((correct / processed) * 100, 14),
                    loss=round(loss.detach().item(), 4),
                )
            )

        test_loss /= len(testloader.dataset)

        return round((correct / processed) * 100, 14), test_loss

In [8]:
import numpy as np


char2idx = config['char2idx']
idx2char = {v:k for k,v in char2idx.items()}
max_sent_len = config['max_sent_len']


def seq2seq(text_list):
    model.eval()

    text_ = [[char2idx[char] for char in query][:max_sent_len] for query in text_list]
    
    text = []
    for t in text_:
        text.append(t + [0] * (max_sent_len - len(t)))
    
    text = torch.from_numpy(np.array(text)).to(torch.int64).to(device)
     
    src_mask = (text != 0).unsqueeze(1)

    decoder_inp = [char2idx["<SOS>"]]

    for i in range(cdata.max_output_len):
        trg_tensor = torch.LongTensor(decoder_inp).unsqueeze(0).to(device)

        trg_mask = (trg_tensor != 0).unsqueeze(-2)
        dimension = trg_tensor.size(-1)  # get seq_len for matrix

        mask = torch.tril(torch.ones(1, dimension, dimension, device=device))
        mask[mask != 0] = 1
        mask = Variable(mask > 0)

        if trg_tensor.is_cuda:
            mask.cuda()
        trg_mask = trg_mask & mask

        with torch.no_grad():
            output = model.forward(text, trg_tensor, src_mask, trg_mask)

        pred_token = output.argmax(2)[:,-1].item()
        decoder_inp.append(pred_token)

        if pred_token == char2idx["<EOS>"]:
            break

    return ''.join([idx2char[i] for i in decoder_inp][1:])
  
# # 'DETTOL ORIGINAL HAND WASH REFILL PACKET 175 ML RECKITT BENCKISER INDIA LIMITED<EOS>'
seq2seq(['Reckitt'.upper()])

In [ ]:
from pathlib import Path

EPOCHS = 200

stats_file = folder_path + 'stats.csv'
stats = pd.read_csv(stats_file).values.tolist() if os.path.exists(stats_file) else []

for epoch in range(checkpoint['epoch']+1, EPOCHS):
    print(seq2seq(['Makhana'.upper()]))
    print(seq2seq(['Pampers'.upper()]))
    print(seq2seq(['crocin'.upper()]))
    print(seq2seq(['hajmola'.upper()]))
    print(seq2seq(['rin'.upper()]))

    # model training
    acc, loss = train(epoch, model, device, trainloader, optimizer)
        
    # model testing
    tacc, tloss = test(model, device, testloader)
    
    stats.append([epoch, acc, loss, tacc, tloss])
    
    torch.save(
        {
            "epoch": epoch,
            "model_state_dict": model.state_dict(), 
            "optimizer_state_dict": optimizer.optimizer.state_dict(),
            "step_num": optimizer.step_num,
        },
        folder_path + "model.pt",
    )
    
    if epoch % 5 == 0:
        path_ = folder_path + "backup/" + str(epoch)
        Path(path_).mkdir(parents=True, exist_ok=True)
        
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.optimizer.state_dict(),
                "step_num": optimizer.step_num,
            },
            path_ + "/model.pt",
        )       
 
    pd.DataFrame(stats, columns=['epoch', 'train_accuracy', 'train_loss', 'test_accuracy', 'test_loss']).to_csv(stats_file, index=False)